## Face Recognition Program - For Mailing, Whatsapp, AWS Service Usage!!


###  Creating Training Data for Face Reconnition using Haarcascade Model

In [ ]:
import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    
    if faces is ():
        return None
    
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

cap = cv2.VideoCapture(0)
count = 0

while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        file_name_path = './faces/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: 
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

### Training My Face Model

In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

data_path = './faces/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

Training_Data, Labels = [], []

for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

Labels = np.asarray(Labels, dtype=np.int32)

model  = cv2.face_LBPHFaceRecognizer.create() 
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")


### Task 6.1 : Mailing While Face Detected

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

def email():
    fromaddr = "2018cscloudakanksha6698@poornima.edu.in"
    toaddr = "aksg530@gmail.com"
    msg = MIMEMultipart()  
    msg['From'] = fromaddr
    msg['To'] = toaddr
    msg['Subject'] = "Sending Akanksha's photo"
    body = "Akanksha's Photo"
    msg.attach(MIMEText(body, 'plain'))  
    filename = "akuu15.jpg"
    attachment = open("./faces/akuu15.jpg", "rb")
    p = MIMEBase('application', 'octet-stream')
    p.set_payload((attachment).read())
    encoders.encode_base64(p)
    p.add_header('Content-Disposition', "attachment; filename= %s" % filename)
    msg.attach(p)
    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.starttls()
    password=input("enter Password: ") 
    s.login(fromaddr, password)
    text = msg.as_string()
    s.sendmail(fromaddr, toaddr, text)
    s.quit()

### Task 6.2 : Send whatsapp message to your friend while face Detected

In [ ]:
import pywhatkit as kit
import datetime
def whatsapp():
    current_time = datetime.datetime.now()
    concode="+91"
    number=input("Enter Number: ")
    kit.sendwhatmsg(" ".join([concode, number]), "It's an automated message from Akanksha(India)", current_time.hour, current_time.minute+1, wait_time=10)

### Task 6.3 : Create EC2 instance using AWS using CLI command

In [ ]:
import os
def AWS_launch():
    itype=input("Enter Instance Type: ")
    key=input("Enter Key Name: ")
    count=input("How many instance you want to launch? : ")
    os.system("aws ec2 run-instances --image-id ami-011c99152163a87ae  --instance-type {itype} --count {count} --subnet-id subnet-718a9e19  --security-group-ids sg-05580d069bf2756bd --key-name {key}".format(key=key, itype=itype, count=count))
    

### Task 6.4 : Create EBS Volume using AWS using CLI command

In [ ]:
def EBS_launch():
    size=input("Input Size of Volume you want to create: ")
    os.system("aws ec2 create-volume --availability-zone ap-south-1a --size {size} --volume-type gp2 --tag-specification  ResourceType=volume,Tags=[{Key=taSK,Value=6}]".format(size=size))
   

###  Task 6.5 : Attach the Newly Created Volume to EC2 instance

In [ ]:
def EBS_attach():
    volume_id=input("Enter volume Id: ")
    instance_id=input("Enter instance Id: ")
    os.system("aws ec2 attach-volume --volume-id {v} --instance-id {i} --device xvdh".format(v = volume_id, i = instance_id))

### Running Facial Recognition Model to predict  face and do particular task

In [ ]:
import cv2
import numpy as np
import os


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        results = model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 80:
            cv2.putText(image, "Hey Akanksha", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            whatsapp_msg_sent()
            AWS_launch()
            EBS_launch()
            EBS_attach()
            # email
            break
         
        else:
            
            cv2.putText(image, "I dont know, how r u", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: 
        break
        
cap.release()
cv2.destroyAllWindows()     